In [1]:
import numpy as np
from scipy.stats import entropy
import pandas as pd

In [4]:
!samtools view data/CLIP-35L33G.bam | head

SRR458758.23028115	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	HHGHHGE@GGHHED?GEGDG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23879202	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	GBGGEEAAF=CEEDEGBGG>	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23893532	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	G@?GGE=EFBDDGGGGD@GG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.26501547	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	IIIHIIIHIGHIIIHEHIHH	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.685997	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	EG7:776DGGGBD<GGEBGEG@	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+
SRR458758.12045118	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	IFGIDDIHIHEIIIIIIIIHBI	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+
SRR458758.15313921	16	chr1	3059006	0	22M	*	0	0	TTCATTTAC

In [5]:
# Position별로 mapping된 base 확인을 위해 mpileup
# base의 비율만 보면 되니까 reference는 따로 필요없음

!samtools mpileup data/CLIP-35L33G.bam > data/CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files


In [6]:
!head data/CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [8]:
# Read count가 50보다 큰 position만 filtering

!awk '$4 > 50 { print $0; }' data/CLIP-35L33G.pileup > data/CLIP-35L33G_filtered.pileup

In [10]:
# pileup file의 크기가 너무 커서, filtering 이전의 file은 삭제

!rm -rf data/CLIP-35L33G.pileup

In [9]:
!head data/CLIP-35L33G_filtered.pileup

chr1	3222722	N	124	CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A	HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GIH@HEIHDIHDIIGHFFGFIIHI@HIGHIIIIIIIIGGBIBHHIHDGIIHBIF=GDDHIHGIHDHIHGBGBFGGHHGGF
chr1	3222723	N	146	GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G	E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GII@@HIHGIIEIIGIH0DIIIHIDHIBGIIIIIIIIIF?IBHHIHDGIIHGIBGEHDEI1HGHHEHFIGB@GH>GHDE>IGHIBHIEGGHGBGHGIGIDII
chr1	3222724	N	155	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A

In [2]:
# pileup file에서 사용할 column만 dataframe으로 불러옴

df = pd.read_csv("data/CLIP-35L33G_filtered.pileup", sep="\t", usecols=[0, 1, 3, 4], names=["chrom", "pos", "count", "basereads"])
df

,chrom,pos,count,basereads
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...


In [3]:
# Read count가 50 넘는 position만 잘 filtering되었는지 확인

(df["count"] > 50).all()

True

In [4]:
# match & substitution만 남김

import re
toremove = re.compile('[<>$*#^]')
df["matches"] = df["basereads"].apply(lambda x: toremove.sub('', x))
df

,chrom,pos,count,basereads,matches
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [6]:
# pileup file의 basereads에서 소문자는 negative strand에 mapping된 것을 뜻한다고 함.
# 하지만 알아서 complementary base로 나타내서 보여준다고 함.
# 그래서 모두 일단 대문자로 바꾸기

df["matches"] = df.matches.str.upper()
df

,chrom,pos,count,basereads,matches
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [7]:
# Count bases per position

for base in ["A", "T", "C", "G"]:
    df[base] = df.matches.str.count(base)

df

,chrom,pos,count,basereads,matches,A,T,C,G
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,121,0,3,0
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0,0,0,146
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,155,0,0,0
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,5,0,0,152
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,55,0,20,81
...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0,301,0,0
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,293,0,0,0
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0,1,284,0
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0,287,0,0


In [8]:
# 합이 1이 되도록 scaling

df[["A", "T", "C", "G"]] = df[["A", "T", "C", "G"]].div(df[["A", "T", "C", "G"]].sum(axis=1), axis=0)
df

,chrom,pos,count,basereads,matches,A,T,C,G
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,0.975806,0.000000,0.024194,0.000000
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.000000,0.000000,0.000000,1.000000
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,0.031847,0.000000,0.000000,0.968153
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231
...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.000000,0.003509,0.996491,0.000000
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000


In [9]:
# Shannon entropy 계산 

prop = df[["A", "T", "C", "G"]].to_numpy()
df["entropy"] = -np.sum(prop * np.log(prop + 1e-9), axis=1)
df

,chrom,pos,count,basereads,matches,A,T,C,G,entropy
0,chr1,3222722,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,0.975806,0.000000,0.024194,0.000000,1.139389e-01
1,chr1,3222723,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.000000,0.000000,0.000000,1.000000,-1.000000e-09
2,chr1,3222724,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000,-1.000000e-09
3,chr1,3222725,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,0.031847,0.000000,0.000000,0.968153,1.411055e-01
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231,9.712127e-01
...,...,...,...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000,-1.000000e-09
24696040,MU069435.1,1565,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,1.000000,0.000000,0.000000,0.000000,-1.000000e-09
24696041,MU069435.1,1566,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.000000,0.003509,0.996491,0.000000,2.333590e-02
24696042,MU069435.1,1567,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,0.000000,1.000000,0.000000,0.000000,-1.000000e-09


In [10]:
# Shannon entropy 0.8 이상인 position filtering

pos_df = df[df.entropy >= 0.8]
pos_df

,chrom,pos,count,basereads,matches,A,T,C,G,entropy
4,chr1,3222726,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,0.352564,0.000000,0.128205,0.519231,0.971213
46,chr1,3224576,168,CCCG*GGGG*GGGGC*GGGGGGGGGG**GGGGGCTCTGCCCCCCCG...,CCCGGGGGGGGGCGGGGGGGGGGGGGGGCTCTGCCCCCCCGCCCGG...,0.000000,0.075949,0.335443,0.588608,0.874139
166,chr1,4329962,55,<<ccccccccccccc$tttc$c$c$ttttc$ttcggcgggcccccc...,CCCCCCCCCCCCCTTTCCCTTTTCTTCGGCGGGCCCCCCCCCCCTT...,0.000000,0.320755,0.584906,0.094340,0.901133
861,chr1,4841534,272,<<<AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAA...,AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAAAAA...,0.537879,0.113636,0.113636,0.234848,1.168064
1058,chr1,4846654,691,<<<ccccccccccccccccccccccccttttttccccctccccccc...,CCCCCCCCCCCCCCCCCCCCCCCCTTTTTTCCCCCTCCCCCCCCCC...,0.004392,0.263543,0.629575,0.102489,0.900067
...,...,...,...,...,...,...,...,...,...,...
24695242,chrM,15317,80,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,0.000000,0.562500,0.212500,0.225000,0.988388
24695339,chrM,15414,90,tt$tttgattttttt$gtt$tgtattttttgattagttt$gat$aa...,TTTTTGATTTTTTTGTTTGTATTTTTTGATTAGTTTGATAAATTTT...,0.388889,0.544444,0.000000,0.066667,0.878844
24695613,MU069435.1,386,217,TT*****GG*G*GGTGTG*G**TG*TG**TTGTTTTG**GGGTCCC...,TTGGGGGTGTGGTGTGTTGTTTTGGGGTCCCGTGTGCGCGGATTGC...,0.032680,0.392157,0.111111,0.464052,1.079309
24695801,MU069435.1,581,2854,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.004205,0.128241,0.229502,0.638052,0.910884


In [11]:
# 해당 position 근처의 20nt sequence를 가져와야 하는데, positive strand와 negative strand를 구별해야 함.
# LIN28A는 single-stranded RNA를 인식해서 붙었을 테니 그 gene이 어느 strand에서 전사된 건지를 구분해야 하는데,
# CLIP-seq의 결과로 읽힌 read가 strand를 구별해서 읽혔다는 보장이 있는지 모르겠음.
# 그래서 GTF file을 확인해보고 맞는 strand에서 떼와야 할 듯.

In [13]:
from gtfparse import read_gtf

gtf = read_gtf("data/gencode.gtf")
gtf = gtf[["seqname", "feature", "start", "end", "strand", "gene_type", "gene_name"]]
gtf

/data/project/danyoung/miniconda3/envs/binfo1/lib/python3.7/site-packages/gtfparse/read_gtf.py:154: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  features=features)
/data/project/danyoung/miniconda3/envs/binfo1/lib/python3.7/site-packages/gtfparse/read_gtf.py:154: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  features=features)
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


,seqname,feature,start,end,strand,gene_type,gene_name
0,chr1,gene,3143476,3144545,+,TEC,4933401J01Rik
1,chr1,transcript,3143476,3144545,+,TEC,4933401J01Rik
2,chr1,exon,3143476,3144545,+,TEC,4933401J01Rik
3,chr1,gene,3172239,3172348,+,snRNA,Gm26206
4,chr1,transcript,3172239,3172348,+,snRNA,Gm26206
...,...,...,...,...,...,...,...
1869102,chrM,transcript,15289,15355,+,Mt_tRNA,mt-Tt
1869103,chrM,exon,15289,15355,+,Mt_tRNA,mt-Tt
1869104,chrM,gene,15356,15422,-,Mt_tRNA,mt-Tp
1869105,chrM,transcript,15356,15422,-,Mt_tRNA,mt-Tp


In [62]:
# gtf file에서 gene에 해당하는 row만 골라냄

gtf = gtf[gtf.feature == "gene"]
gtf

,seqname,feature,start,end,strand,gene_type,gene_name
0,chr1,gene,3143476,3144545,+,TEC,4933401J01Rik
3,chr1,gene,3172239,3172348,+,snRNA,Gm26206
6,chr1,gene,3276124,3741721,-,protein_coding,Xkr4
24,chr1,gene,3322980,3323459,+,processed_pseudogene,Gm18956
27,chr1,gene,3435954,3438772,-,TEC,Gm37180
...,...,...,...,...,...,...,...
1869086,chrM,gene,13552,14070,-,protein_coding,mt-Nd6
1869093,chrM,gene,14071,14139,-,Mt_tRNA,mt-Te
1869096,chrM,gene,14145,15288,+,protein_coding,mt-Cytb
1869101,chrM,gene,15289,15355,+,Mt_tRNA,mt-Tt


In [64]:
# position 중에 gtf file에 annotation이 안 된 position이 몇 개나 되는지 확인
from tqdm import tqdm

n = 0
for i in tqdm(range(len(pos_df))):
    ex = pos_df.iloc[i]
    corr_gtf = gtf[(ex.chrom == gtf.seqname) & (gtf.start < ex.pos) & (ex.pos < gtf.end)]
    if len(corr_gtf) == 0:
        n += 1
print(n)    

100%|██████████| 14901/14901 [00:49<00:00, 298.04it/s]

356


In [73]:
# 356개밖에 안 되니, 그냥 제외하자.

def gtf_has_annot(row):
    if ((row.chrom == gtf.seqname) & (gtf.start < row.pos) & (row.pos < gtf.end)).sum() != 0:
        return True
    else:
        return False
    
pos_df = pos_df[pos_df.apply(gtf_has_annot, axis=1)]
pos_df

,chrom,pos,count,basereads,matches,A,T,C,G,entropy
166,chr1,4329962,55,<<ccccccccccccc$tttc$c$c$ttttc$ttcggcgggcccccc...,CCCCCCCCCCCCCTTTCCCTTTTCTTCGGCGGGCCCCCCCCCCCTT...,0.000000,0.320755,0.584906,0.094340,0.901133
861,chr1,4841534,272,<<<AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAA...,AGAAGAAAAGGAAGAAAAAAAGAAAAGGATTTTTAAAAAAAAAAAA...,0.537879,0.113636,0.113636,0.234848,1.168064
1058,chr1,4846654,691,<<<ccccccccccccccccccccccccttttttccccctccccccc...,CCCCCCCCCCCCCCCCCCCCCCCCTTTTTTCCCCCTCCCCCCCCCC...,0.004392,0.263543,0.629575,0.102489,0.900067
6624,chr1,4854324,626,ccccccctcctcccccccctcccctcgcccgcgcggccg*cc**cg...,CCCCCCCTCCTCCCCCCCCTCCCCTCGCCCGCGCGGCCGCCCGGCC...,0.012448,0.145228,0.719917,0.122407,0.828493
8125,chr1,4878699,110,AAGGGGGGGGGGGGGGGGCCAAGGAAGAGCCAAAACGAGAGGGGGG...,AAGGGGGGGGGGGGGGGGCCAAGGAAGAGCCAAAACGAGAGGGGGG...,0.372727,0.000000,0.109091,0.518182,0.950214
...,...,...,...,...,...,...,...,...,...,...
24694030,chrM,5249,128,>>>>>cgcctaaccccctacacccccccccccatccactccgcacc...,CGCCTAACCCCCTACACCCCCCCCCCCATCCACTCCGCACCCCCCC...,0.089431,0.081301,0.756098,0.073171,0.822676
24694139,chrM,9819,53,TTTTTTTTTTTTTT^HT^GA^IT^HA^HA^GA^GA^DA^HA^IA^H...,TTTTTTTTTTTTTTHTGAITHAHAGAGADAHAIAHAHAITITIAHT...,0.491803,0.377049,0.000000,0.131148,0.983205
24694297,chrM,11706,79,AAAAAAAAAAGGGGGGGGGGGGGGGGGGGgggggggggggGGGGGG...,AAAAAAAAAAGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.126582,0.164557,0.000000,0.708861,0.802487
24695242,chrM,15317,80,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,GGGGGTCTTTTTTCTTTGGTCCTTTTGTTCGTTTTTTGTGTTCTCT...,0.000000,0.562500,0.212500,0.225000,0.988388


In [85]:
from Bio import SeqIO

for record in SeqIO.parse("data/Mus_musculus.GRCm39.dna.primary_assembly.fa", "fasta"):
    print(record.id)

1
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
MT
X
Y
JH584299.1
GL456233.2
JH584301.1
GL456211.1
GL456221.1
JH584297.1
JH584296.1
GL456354.1
JH584298.1
JH584300.1
GL456219.1
GL456210.1
JH584303.1
JH584302.1
GL456212.1
JH584304.1
GL456379.1
GL456366.1
GL456367.1
GL456239.1
GL456383.1
GL456385.1
GL456360.1
GL456378.1
MU069435.1
GL456389.1
GL456372.1
GL456370.1
GL456381.1
GL456387.1
GL456390.1
GL456394.1
GL456392.1
GL456382.1
GL456359.1
GL456396.1
GL456368.1
MU069434.1
JH584295.1


In [89]:
# dataframe의 chromosome name과 fasta file의 chromosome 이름을 통일시키기
pos_df["chrom"] = pos_df.chrom.str.split("chr").str[1]
pos_df.chrom.unique()

/data/project/danyoung/miniconda3/envs/binfo1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', 'X', 'Y', 'M'],
      dtype=object)

In [90]:
pos_df["chrom"] = pos_df.chrom.map(lambda x: "MT" if x == "M" else x)
pos_df.chrom.unique()

/data/project/danyoung/miniconda3/envs/binfo1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', 'X', 'Y', 'MT'],
      dtype=object)

In [97]:
gtf["seqname"] = gtf.seqname.str.split("chr").str[1]
gtf["seqname"] = gtf.seqname.map(lambda x: "MT" if x == "M" else x)
gtf.seqname.unique()

/data/project/danyoung/miniconda3/envs/binfo1/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/data/project/danyoung/miniconda3/envs/binfo1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', 'X', 'Y', 'MT'],
      dtype=object)

In [128]:
# 이제, pos_df의 각 position별로 flanking 20nt를 reference genome fasta file에서 가져온다. 좌 10nt, 우 10nt
# gtf를 확인해서, 만약 negative strand에 gene이 있다면 reverse complement를 해서 가져온다.

def get_strand(row):
    return gtf[(row.chrom == gtf.seqname) & (gtf.start < row.pos) & (row.pos < gtf.end)].iloc[0].strand

def get_flanking_seq(row, seq):
    pos = row.pos
    strand = get_strand(row)
    motif = seq[pos-11: pos+10]
    if strand == "-":
        motif = motif.reverse_complement()
        
    return motif

def get_motifs(chrom, seq):
    motifs = []
    chrom_df = pos_df[pos_df.chrom == chrom]
    for _, row in chrom_df.iterrows():
        motifs.append(get_flanking_seq(row, seq))
    
    return motifs
        
def get_all_motifs(ref):
    motifs = []
    for record in SeqIO.parse(ref, "fasta"):
        chrom = record.id
        if chrom not in pos_df.chrom.unique():
            continue
        seq = record.seq
        chrom_motifs = get_motifs(chrom, seq)
        
        motifs = motifs + chrom_motifs
    
    return motifs

In [130]:
motifs = get_all_motifs("data/Mus_musculus.GRCm39.dna.primary_assembly.fa")

In [137]:
motifs = list(map(str, motifs))
len(motifs)

14545

- 여기까지 해서 positive sample이 될 sequence들을 얻었고,
- 이제 negative sample을 얻어야 함.
    - 시간 관계상 randomly generated sequence로 하기로 함.
    - 개수는 positive sample과 똑같이
    - 다만, positive sample과 겹치지 않도록!

In [157]:
import random

def random_seq(length):
    seq = "".join([random.choice("ATGC") for _ in range(length)])
    return seq

In [178]:
negs = [random_seq(21) for _ in range(len(motifs))]
sum(list(map(lambda x: x in motifs, negs)))

0

In [180]:
samples = motifs + negs
len(samples)

29090

In [182]:
labels = [1] * len(motifs) + [0] * len(negs)
len(labels)

29090

In [184]:
data = pd.DataFrame({"seq": samples, "target": labels})
data

,seq,target
0,AGAGTGCCATGGAGGACCTGC,1
1,CCAGTATGACGGTGCATACAA,1
2,TTGTGAGCCAGACGGGCTGCA,1
3,TTGTAGGAAAGACGTCCAAGA,1
4,TCACGGATTGGGAGATACAGG,1
...,...,...
29085,TTGCCCTAAAAATGCGGTATA,0
29086,ATCCACGCCGGGGCTGGCGCG,0
29087,GATCTGGGATTCCGGGAATAG,0
29088,CTTCGGAGAGGCAAAGTTATG,0


In [190]:
from sklearn.model_selection import train_test_split

train_test_split(data, test_size=0.2)

[                         seq  target
 23123  GACATGACACTCCTTCAGTCG       0
 9252   GACCTCACTTGGGTGGGGGAC       1
 9413   GCTTTATAATGTGTTTTAAAG       1
 1725   TAATTATTAAGGGAACTGAAT       1
 16804  TCTTGGATCATACTGGGAGTC       0
 ...                      ...     ...
 22948  GGGCTTCCTGACTGCTCACCG       0
 978    TTGCCACCAAGATGGGTTTTG       1
 12550  ATCTGTGGAAGAGCAGGTTAG       1
 22533  GCAACAATTCATCTGCGGATA       0
 24345  TAGCGAATTCCCATTGGCAAC       0
 
 [23272 rows x 2 columns],
                          seq  target
 19751  AGGAGAGCTAACCGGTGGCGG       0
 17320  ATGGGTTAATCAAAGCGCTTG       0
 13292  GTGATATCAAGTTGGTTGTAA       1
 4795   ATGTACAGAAGGTGGTTGGCT       1
 10248  TCTCGTAGTAGAGAGGTTTCT       1
 ...                      ...     ...
 4071   CAACTCCATTGTGTGGAGGAT       1
 21312  AGCAAGTCCGTAGGGCCATTA       0
 21826  CTGGTAGCGTGTCTCGACACG       0
 19188  GCAGGTGAATGTCGCTACCTA       0
 4115   TGGCTCATGTGAATCCCCAGT       1
 
 [5818 rows x 2 columns]]

In [185]:
data.to_csv("data/dataset.csv", index=False)